In [ ]:
import os

import pandas as pd

In [ ]:
root = 'Export_csv/b71e463_UKE_yeojo_60_median_3_mit_Labor'

In [ ]:
root = 'Export_csv/d860af6_UKE_yeojo_60_median_3_ohne_Labor'

In [ ]:
sg_df = pd.read_csv(root + '/saliency_validset_LONG_.csv', index_col=0)
sg_df = sg_df.append(pd.read_csv(root + '/saliency_testset_LONG_.csv', index_col=0))
sg_df = sg_df.append(pd.read_csv(root + '/MIMIC/saliency_baseset_LONG_.csv', index_col=0))
sg_df = sg_df.append(pd.read_csv(root + '/eICU/saliency_baseset_LONG_.csv', index_col=0))

In [ ]:
ig_df = pd.read_csv(root + '/saliency_validset_LONG_ig.csv', index_col=0)
ig_df = ig_df.append(pd.read_csv(root + '/saliency_testset_LONG_ig.csv', index_col=0))
ig_df = ig_df.append(pd.read_csv(root + '/MIMIC/saliency_baseset_LONG_ig.csv', index_col=0))
ig_df = ig_df.append(pd.read_csv(root + '/eICU/saliency_baseset_LONG_ig.csv', index_col=0))

In [ ]:
targets = pd.read_csv(root + '/validset_targets.csv', index_col=0)
targets = targets.append(pd.read_csv(root + '/validset_targets.csv', index_col=0))
targets = targets.append(pd.read_csv(root + '/MIMIC/baseset_targets.csv', index_col=0))
targets = targets.append(pd.read_csv(root + '/eICU/baseset_targets.csv', index_col=0))

In [ ]:
targets.shape

In [ ]:
sg_df.shape

In [ ]:
ig_df.shape

In [ ]:
ig_df = ig_df.dropna()
sg_df = sg_df.dropna()

In [ ]:
ig_df.shape

In [ ]:
# sample points
import numpy as np
idcs = np.random.choice(range(len(ig_df)), 100000)
ig_df_s = ig_df.iloc[idcs]
sg_df_s = sg_df.iloc[idcs]

In [ ]:
umap_df = sg_df_s.copy()

In [ ]:
# fix PatID:
ids = umap_df["PatID"].astype(str)
ids = [i[7:-2] if i.startswith("tensor") else i for i in ids]
umap_df["PatID"] = pd.Series(ids).astype(float).astype(int)

In [ ]:
import umap
import umap.plot
reducer = umap.UMAP()

umap_cols = [col for col in umap_df.columns if col != "PatID"]
embeddings = reducer.fit_transform(umap_df[umap_cols])

In [ ]:
# get top ten feats
feat_avg = umap_df[umap_cols].abs().mean().sort_values(ascending=False)
feat_avg = feat_avg / sum(feat_avg)

top_ten_feats = feat_avg.iloc[:10].index
top_ten_feats

In [ ]:
high_d_reducer = umap.UMAP(n_components=10,
                           n_neighbors=30,
                           min_dist=0.0,)
high_d_embeddings = high_d_reducer.fit_transform(umap_df[umap_cols])

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
kmeans_labels = kmeans.fit_predict(high_d_embeddings)
umap.plot.points(reducer, labels=kmeans_labels, theme="fire")

In [ ]:
def create_ranking(df, rank_type="abs"):
    # take mean of abs vals over all steps
    #print(df.iloc[0])
    if rank_type == "abs":
        df = df.abs().mean()
    elif rank_type == "pos":
        df = df[df >= 0].mean()
    elif rank_type == "neg":
        df = df[df <= 0].mean()
    #print(df.iloc[0])
    # divide by sum of abs vals to have distr
    df = df / df.abs().sum()
    df = df.transpose()
    df = df.sort_values(ascending=False)
    return df

In [ ]:
def take_top_vals(df):
    df = df.sort_values(ascending=True)
    df = df / df.abs().sum()
    df = df[df.cumsum() > 0.2]
    df = df / df.abs().sum()
    return df

In [ ]:
def plot_ranking(df, ax, rank_type="abs"):
    if rank_type == "abs":
        c = "blue"
    elif rank_type == "pos":
        c = "orange"
    elif rank_type == "neg":
        c = "green"
    df.index = ["".join(idx.split("_")[1:]) if "_" in idx else idx for idx in df.index]
    df.plot.barh(ax=ax, color=c)
    

In [ ]:
def visualize_ranking(df, title, ax=None):
    
    'WARNING'
    #TODO: !!! Need to balance scale of pos and neg according to how many pos and neg cases there were - cannot just add them in one plot as below!
    
    # norm by pat
    #df = df.groupby("PatID").apply(lambda x: x / x.abs().sum()).drop(columns=["PatID"])
    df = df.drop(columns=["PatID"])
    
    ranking_abs = create_ranking(df, rank_type="abs")
    ranking_pos = create_ranking(df, rank_type="pos")
    ranking_neg = create_ranking(df, rank_type="neg")

    top_vals_abs = take_top_vals(ranking_abs)
    ranking_pos = ranking_pos.loc[top_vals_abs.index]
    ranking_neg = ranking_neg.loc[top_vals_abs.index]
    
    if ax is None:
        fig = plt.figure(figsize=(1, 3), dpi=200)
        ax = plt.gca()
    plot_ranking(ranking_pos, ax, rank_type="pos")
    plot_ranking(ranking_neg, ax, rank_type="neg")

        
    ax.set_title(title)


In [ ]:
from hdbscan import HDBSCAN
hdbscan = HDBSCAN(min_cluster_size=2000, min_samples=500)
hdbscan_labels = hdbscan.fit_predict(high_d_embeddings)
print(hdbscan.labels_.max(), (hdbscan.labels_ == -1).mean())
umap.plot.points(reducer, labels=hdbscan_labels, theme="fire")


import matplotlib.pyplot as plt
plt.figure()
cluster_df = umap_df.copy()
cluster_df["cluster"] = hdbscan_labels
cluster_df = cluster_df[cluster_df["cluster"] != -1]
#cluster_ranking = cluster_df.groupby("cluster").apply(lambda x : create_ranking(x))
#plt.close()

#plt.figure(figsize=(5, 3), dpi=200)
num_clusters = cluster_df["cluster"].max() + 1
fig, axs = plt.subplots(1, num_clusters, squeeze=True, figsize=(2 * num_clusters, 4), dpi=25 * num_clusters)
axs = axs.squeeze()
for i in range(num_clusters):
    ax = axs[i]
    current_cluster_df = cluster_df[cluster_df["cluster"] == i].drop(columns=["cluster"])
    #current = cluster_ranking.iloc[i].drop(columns="cluster")  #cluster_df[cluster_df["cluster"] == i]
    title = f"{i}: Size={len(current_cluster_df)} "
    ax = visualize_ranking(current_cluster_df, title, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
from hdbscan import HDBSCAN
hdbscan = HDBSCAN(min_cluster_size=2000, min_samples=500)
hdbscan_labels = hdbscan.fit_predict(high_d_embeddings)
print(hdbscan.labels_.max(), (hdbscan.labels_ == -1).mean())
umap.plot.points(reducer, labels=hdbscan_labels, theme="fire")


import matplotlib.pyplot as plt
plt.figure()
cluster_df = umap_df.copy()
cluster_df["cluster"] = hdbscan_labels
cluster_df = cluster_df[cluster_df["cluster"] != -1]
#cluster_ranking = cluster_df.groupby("cluster").apply(lambda x : create_ranking(x))
#plt.close()

#plt.figure(figsize=(5, 3), dpi=200)
num_clusters = cluster_df["cluster"].max() + 1
fig, axs = plt.subplots(1, num_clusters, squeeze=True, figsize=(2 * num_clusters, 4), dpi=25 * num_clusters)
axs = axs.squeeze()
for i in range(num_clusters):
    ax = axs[i]
    current_cluster_df = cluster_df[cluster_df["cluster"] == i].drop(columns=["cluster"])
    #current = cluster_ranking.iloc[i].drop(columns="cluster")  #cluster_df[cluster_df["cluster"] == i]
    title = f"{i}: Size={len(current_cluster_df)} "
    ax = visualize_ranking(current_cluster_df, title, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
embeddings = reducer.fit_transform(df[top_ten_feats])

In [ ]:
umap.plot.points(reducer, theme='fire')

In [ ]:
umap.plot.points(reducer, theme='fire')